# Make DESI photo-z tables for VO

I was originally going to make a smaller VO table with only:
    
- field
- desi_id
- ra
- dec
- hp_idx
- redshift
- redshift_err
- zspec
 

The HELP field and hp_idx are made in this notebook.

We later decided it would be advantageous to include all columns. Given the large size of the tables some of the work must be done with stilts.

In [1]:
from astropy.table import Table, Column

from herschelhelp.utils import position_to_field
from herschelhelp_internal.utils import coords_to_hpidx

## 1. Make csv file with corrects columns

In [2]:
#Load a smaller version with just required fields made using Stilts
#stilts tpipe cmd='keepcols "ID RA DEC photo_z photo_zerr spec_z"' 
#./data/dr6dr7_photoz_csp_totcat.fits omode=out out=./data/dr6dr7_reduced.fits

small_tab='./data/LS_DR8_total_csp_reduced.fits'
desi = Table.read(small_tab)['ID', 'RA', 'DEC'] #, 'photo_z', 'photo_zerr', 'spec_z']
#desi = Table.read(full_tab)

In [3]:
#desi['RA'].name = 'ra'
#desi['DEC'].name = 'dec'
#desi['ID'].name = 'desi_id'
#desi['photo_z'].name = 'redshift'
#desi['photo_zerr'].name = 'redshift_err'
#desi['spec_z'].name = 'zspec'

In [4]:
desi.add_column(Column(
    data=coords_to_hpidx(desi['RA'], desi['DEC'], order=13),
    name="hp_idx"
))

In [5]:
#desi.write('./data/desi_for_vo_1.fits', overwrite = True)

In [6]:
desi.add_column(Column(
    data=position_to_field(desi['RA'], desi['DEC'], dmu_loc='../../'),
    name="field"
))

In [7]:
#cols = ['field', 'desi_id', 'ra', 'dec', 'hp_idx', 'redshift', 'redshift_err', 'zspec']
#cols = ['field', 'ID', 'RA', 'DEC', 'hp_idx', 'photo_z', 'photo_zerr', 'spec_z']
desi['field', 'hp_idx'].write('./data/desi_for_vo_temp.fits', overwrite = True)

In [ ]:
#You then need to stack the tables using stilts with:
#stilts tjoin nin=2 in1=LS_DR8_total_csp.fits in2=desi_for_vo_temp.fits out=combined.csv